# Computation

In [1]:
#import libraries cell

import json
import time
import findspark
import pyspark

import numpy as np
import pandas as pd

from kafka                import KafkaProducer
from kafka.admin          import KafkaAdminClient, NewTopic
 
from pyspark.sql           import functions as F
from pyspark.sql           import SparkSession
from pyspark.streaming     import StreamingContext
from pyspark.sql.types     import StructField, StructType, DoubleType, IntegerType
from pyspark.sql.functions import from_json, col, when, count, struct, collect_list
from pyspark               import SparkConf, SparkContext
from pyspark.sql.types import StringType 

## Spark setup

In [2]:
#initialisation of spark from the packages folder
findspark.init('/usr/local/spark')

In [3]:
#start session - specify port, application name, and configuration settings.

spark = SparkSession.builder\
        .appName("Project_CosmicRays_Dashboard_application")\
        .master("spark://master:7077")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")\
        .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")\
        .config("spark.ui.port", "4041")\
        .getOrCreate()

spark

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aidin/.ivy2/cache
The jars for the packages stored in: /home/aidin/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-092b09ad-3c8b-4c4f-94e2-8bb4ec9be3a0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 810ms :: artifacts dl 27ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central i

22/09/16 22:33:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/16 22:33:34 WARN StandaloneAppClient$ClientEndpoint: Failed to connect to master master:7077
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anon$1.run(StandaloneAppClient.scala:107)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
Caused by: java.io.IOException: Failed to 

22/09/16 22:34:13 WARN StandaloneAppClient$ClientEndpoint: Failed to connect to master master:7077
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anon$1.run(StandaloneAppClient.scala:107)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
Caused by: java.io.IOException: Failed to 

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: Can only call getServletHandlers on a running MetricsSystem
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.metrics.MetricsSystem.getServletHandlers(MetricsSystem.scala:89)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:650)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngi

## Kafka Setup

In [4]:
KAFKA_BOOTSTRAP_SERVERS = 'pd-slave3:9092'

producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)
kafka_admin = KafkaAdminClient(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [5]:
kafka_admin.list_topics()

['topic_results', 'topic_stream', '__consumer_offsets']

In [6]:
# input dataframe and its source. 
# Define subscription to 'topic_stream' - one of the two topics in kafka

inputDF = spark.readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
        .option('subscribe', 'topic_stream')\
        .load()

inputDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# scheme of the imput data
schema = StructType(
        [
                StructField("HEAD",        StringType()),
                StructField("FPGA",        StringType()),
                StructField("TDC_CHANNEL", StringType()),
                StructField("ORBIT_CNT",   StringType()),
                StructField("BX_COUNTER",  StringType()),
                StructField("TDC_MEAS",    StringType())
        ]
    )

In [8]:
# convert inputDF to jsonDF

jsonDF = inputDF.select(from_json(col("value").alias('value').cast("string"), schema).alias('value'))

jsonDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- HEAD: string (nullable = true)
 |    |-- FPGA: string (nullable = true)
 |    |-- TDC_CHANNEL: string (nullable = true)
 |    |-- ORBIT_CNT: string (nullable = true)
 |    |-- BX_COUNTER: string (nullable = true)
 |    |-- TDC_MEAS: string (nullable = true)



In [9]:
flatDF = jsonDF.select("value.HEAD", 
                       "value.FPGA", 
                       "value.TDC_CHANNEL",
                       "value.ORBIT_CNT",
                       "value.BX_COUNTER",
                       "value.TDC_MEAS")

flatDF.printSchema()

root
 |-- HEAD: string (nullable = true)
 |-- FPGA: string (nullable = true)
 |-- TDC_CHANNEL: string (nullable = true)
 |-- ORBIT_CNT: string (nullable = true)
 |-- BX_COUNTER: string (nullable = true)
 |-- TDC_MEAS: string (nullable = true)



In [10]:
# data-cleansing
df = flatDF.where(col("HEAD")==2)

# division of dataframe between chambers
df = df.withColumn('CHAMBER',
        when(( col( 'FPGA' ) == 0 ) & ( col( 'TDC_CHANNEL' ) >=  0 ) & ( col( 'TDC_CHANNEL' ) <  64  ), 0) \
       .when(( col( 'FPGA' ) == 0 ) & ( col( 'TDC_CHANNEL' ) >= 64 ) & ( col( 'TDC_CHANNEL' ) <= 127 ), 1) \
       .when(( col( 'FPGA' ) == 1 ) & ( col( 'TDC_CHANNEL' ) >=  0 ) & ( col( 'TDC_CHANNEL' ) <  64  ), 2) \
       .when(( col( 'FPGA' ) == 1 ) & ( col( 'TDC_CHANNEL' ) >= 64 ) & ( col( 'TDC_CHANNEL' ) <= 127 ), 3) \
       .when(( col( 'FPGA' ) == 1 ) &                                  ( col( 'TDC_CHANNEL' ) == 128 ), 4) \
               .otherwise(None))                                                                           \
      .filter( col( 'CHAMBER' ).isNotNull() )

df.printSchema()

root
 |-- HEAD: string (nullable = true)
 |-- FPGA: string (nullable = true)
 |-- TDC_CHANNEL: string (nullable = true)
 |-- ORBIT_CNT: string (nullable = true)
 |-- BX_COUNTER: string (nullable = true)
 |-- TDC_MEAS: string (nullable = true)
 |-- CHAMBER: integer (nullable = true)



In [11]:
# scintillator time offset by Chamber
# TODO compute ABSOLUTETIME and DRIFTIME
time_offset_by_chamber = {
0: 95.0 - 1.1, # Ch 0
1: 95.0 + 6.4, # Ch 1
2: 95.0 + 0.5, # Ch 2
3: 95.0 - 2.6, # Ch 3
}

In [12]:
msg_json = {
    'epoch_id' : 0,
    'hits'     : 0,
    'CH0' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            }
            },
    'CH1' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            }
            },
    'CH2' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            }
            },
    'CH3' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            }
            },
}

In [13]:
def batch_proc(df, epoch_id):   

    '''
    Function for the batch processing of the data.
    The processing consists in retrieving
    the following informations:
        1. total number of processed hits, 
           post-clensing (1 value per batch)
        2. total number of processed hits,
           post-clensing, per chamber (4 values per batch)
        3. histogram of the counts of active TDC_CHANNEL,
           per chamber (4 arrays per batch)
        4. histogram of the total number of active
           TDC_CHANNEL in each ORBIT_CNT, per chamber 
           (4 arrays per batch)
        5. histogram of the counts of active TDC_CHANNEL,
           per chamber, ONLY for those orbits with at least
           one scintillator signal in it (4 arrays per batch)
        6. histogram of the DRIFTIME, per chamber 
           (4 arrays per batch) TODO
    
    Inputs:
        - df: spark dataframe with the data
        - epoch_id: batch index    
    '''

    msg_json['epoch_id']    = epoch_id

    # total number of processed hits
    # post-cleansing
    total_hits = df.filter(col('CHAMBER') != 4)\
                   .count()


    # total number of processed hits,
    # post-clensing, per chamber
    df_counts = df.filter(col('CHAMBER') != 4) \
                  .groupBy(   'CHAMBER')       \
                  .count()                     \
                  .withColumnRenamed('count', 
                                     'ch_hits')


    # histogram of the counts of active
    # TDC_CHANNEL, per chamber.
    df_counts_ch = df.filter(col('CHAMBER') != 4)            \
                     .groupBy('TDC_CHANNEL', 'CHAMBER')      \
                     .count()                                \
                     .groupBy('CHAMBER')                     \
                     .agg(struct(collect_list('TDC_CHANNEL') \
                                       .alias(  'bin_edges'),
                                 collect_list(      'count') \
                                       .alias(     'counts'),
                                )                            \
                          .alias('histo_CH')
                         )
    
    # histogram of the total number of active
    # TDC_CHANNEL in each ORBIT_CNT per chamber
    df_counts_orb = df.filter(col('CHAMBER') != 4)                    \
                      .groupBy('ORBIT_CNT', 'CHAMBER')                \
                      .agg(F.countDistinct("TDC_CHANNEL"))            \
                      .groupBy('count(TDC_CHANNEL)', 'CHAMBER')       \
                      .count()                                        \
                      .groupBy('CHAMBER')                             \
                      .agg(struct(collect_list('count(TDC_CHANNEL)')  \
                                        .alias(         'bin_edges'),
                                  collect_list(             'count') \
                                        .alias(            'counts'),
                                 )                                    \
                           .alias('histo_ORB')
                          )


    # Histogram of the count of active TDC_CHANNEL,
    # per chamber, only for those orbits with
    # at least one scintillatorin it
    
    # revision needed: not sure about this
    list_scint = df.filter(col('CHAMBER') == 4)   \
                   .select('ORBIT_CNT')           \

    df_counts_sc = df.filter(col('CHAMBER') != 4 )                  \
                     .join(list_scint, on="ORBIT_CNT", how="inner") \
                     .groupBy(  'TDC_CHANNEL', 'CHAMBER'       )    \
                     .count()                                       \
                     .groupBy('CHAMBER')                            \
                     .agg(struct(collect_list('TDC_CHANNEL')        \
                                         .alias('bin_edges'),
                                 collect_list(      'count')        \
                                         .alias(   'counts'))       \
                          .alias('histo_SC')
                         )

    # dataframe with results joined
    df_res = df_counts.join(df_counts_ch,  on='CHAMBER') \
                      .join(df_counts_orb, on='CHAMBER') \
                      .join(df_counts_sc,  on='CHAMBER') \
                      .sort('CHAMBER')                   \
                      .collect()
                      
    msg_json['hits'] = total_hits
    for i in range(len(df_res)):
        msg_json[f'CH{i}']['total_hits']               = df_res[i][  'ch_hits']
        msg_json[f'CH{i}'][  'histo_CH'][ 'bin_edges'] = df_res[i][ 'histo_CH']['bin_edges']
        msg_json[f'CH{i}'][  'histo_CH']['bin_counts'] = df_res[i][ 'histo_CH'][   'counts']
        msg_json[f'CH{i}'][ 'histo_ORB'][ 'bin_edges'] = df_res[i]['histo_ORB']['bin_edges']
        msg_json[f'CH{i}'][ 'histo_ORB']['bin_counts'] = df_res[i]['histo_ORB'][   'counts']
        msg_json[f'CH{i}'][  'histo_SC'][ 'bin_edges'] = df_res[i][ 'histo_SC']['bin_edges']
        msg_json[f'CH{i}'][  'histo_SC']['bin_counts'] = df_res[i][ 'histo_SC'][   'counts']

    producer.send('topic_results', json.dumps(msg_json).encode('utf-8'))
    producer.flush()
    pass

In [14]:
df.isStreaming

True

In [ ]:
df.writeStream\
    .foreachBatch(batch_proc)\
    .start()\
    .awaitTermination()

22/09/16 22:31:52 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a5fa423b-e160-460a-a30a-255405cc9312. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/09/16 22:31:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
spark.stop()